In [1]:
import os, sys, pandas as pd, glob
import datetime, time
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None)

In [3]:
input_dir = "../sample_data/iot_data/*.csv"
input_data_file_list = glob.glob(input_dir)

li = []
for filename in input_data_file_list:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
# filter out the lines from the 
df = df[~pd.isna(df["pr"])]


In [5]:
def getTimestamp(time_str):
    return time.mktime(datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S").timetuple())
    
df['st_timestamp'] = df['ts'].apply(getTimestamp)
df['et_timestamp'] = df['te'].apply(getTimestamp)

In [77]:
df_mac2ip = pd.read_csv('data/iot_mac2ip.csv', sep='\t')
df_mac2ip['fname'] = df_mac2ip['MAC'].apply(lambda x: x.replace(':', '') + '.csv')
mac2ip_list = df_mac2ip.values.tolist()
dfTemp = df
dfTemp['flowTag'] = ""

[['ec:1a:59:83:28:11', '192.168.1.165', 'WEMO Motion Sensor', 'ec1a59832811.csv'], ['ec:1a:59:79:f4:89', '192.168.1.223', 'WEMO Power Switch', 'ec1a5979f489.csv'], ['00:16:6c:ab:6b:88', '192.168.1.248', 'Samsung Camera', '00166cab6b88.csv'], ['50:c7:bf:00:56:39', '192.168.1.227', 'TP Link Plug', '50c7bf005639.csv'], ['70:ee:50:18:34:43', '192.168.1.241', 'Netatmo Camera', '70ee50183443.csv'], ['00:17:88:2B:9A:25', '192.168.1.129', 'Huebulb', '0017882B9A25.csv'], ['44:65:0D:56:CC:D3', '192.168.1.239', 'AmazonEcho', '44650D56CCD3.csv'], ['F4:F5:D8:8F:0A:3C', '192.168.1.119', 'chromecast', 'F4F5D88F0A3C.csv'], ['74:C6:3B:29:D7:1D', '192.168.1.163', 'ihome', '74C63B29D71D.csv'], ['D0:73:D5:01:83:08', '192.168.1.118', 'lifx', 'D073D5018308.csv']]


In [88]:
for i in range(0, len(mac2ip_list)):
    fname = '../data/annotations/' + mac2ip_list[i][3]
    ip_addr = mac2ip_list[i][1]
    print(fname)
    df_annotation = pd.read_csv(fname)
    l = df_annotation.values.tolist()
    for i in range(0, len(l)):
        v = l[i]
        (st, et) = (v[0], v[1])
        tag = v[2]
        dfTemp['flowTag'].loc[(df['st_timestamp'] >= st) & (df['st_timestamp'] < et) & ((df['sa'] == ip_addr) | (df['da'] == ip_addr)) ] = tag

    


../data/annotations/ec1a59832811.csv
../data/annotations/ec1a5979f489.csv
../data/annotations/00166cab6b88.csv
../data/annotations/50c7bf005639.csv
../data/annotations/70ee50183443.csv
../data/annotations/0017882B9A25.csv
../data/annotations/44650D56CCD3.csv
../data/annotations/F4F5D88F0A3C.csv
../data/annotations/74C63B29D71D.csv
../data/annotations/D073D5018308.csv


In [89]:
print(dfTemp[dfTemp['flowTag'] != '']['flowTag'])

Series([], Name: flowTag, dtype: object)


In [73]:
dfTemp[dfTemp['flowTag'] != '']['flowTag']

Series([], Name: flowTag, dtype: object)

In [9]:
ips_to_remap_list = ["192.168.1.1", "192.168.1.2"]
ips_to_remap_dict = dict.fromkeys(ips_to_remap_list, 1)

In [38]:
## try with the brute-force attack

                        ts                   te       td                   sa  \
0      2018-10-26 06:00:25  2018-10-26 06:04:35  250.011          192.168.1.1   
1      2018-10-26 06:04:36  2018-10-26 06:04:36    0.097        192.168.1.119   
2      2018-10-26 05:55:49  2018-10-26 06:00:20  270.818       74.125.200.188   
3      2018-10-26 05:59:26  2018-10-26 05:59:29    2.859       174.129.217.97   
4      2018-10-26 06:02:20  2018-10-26 06:02:20    0.000        192.168.1.108   
...                    ...                  ...      ...                  ...   
99311  2018-10-26 06:28:34  2018-10-26 06:28:41    6.694       174.129.217.97   
99312  2018-10-26 06:26:37  2018-10-26 06:26:37    0.000        192.168.1.119   
99313  2018-10-26 06:26:41  2018-10-26 06:26:41    0.042  fdc1:1bdc:1e84::911   
99314  2018-10-26 06:26:37  2018-10-26 06:26:37    0.008        192.168.1.129   
99315  2018-10-26 06:23:56  2018-10-26 06:28:26  269.997        192.168.1.118   

                      da   

### Remapping function

In [12]:
#gets a list of IP addresses to remap 
def get_ip_pool(fname):
    df = pd.read_csv(fname, header=0)
    ip_list = list(df["ip"])
    return ip_list

# ip_list = get_ip_pool("data/ip_list.csv")
ip_list=ips_to_remap_list
last_remap_idx = 0
len_ip_list = len(ip_list)
flow_map = {}

# remap the IP address. Don't parallelize the function
def remap_ip(x, field_name):
    src_ip, src_port, dst_ip, dst_port = (x["sa"], x["sp"], x["da"], x["dp"])
    ip_to_map = src_ip
    if field_name == "da":
        ip_to_map = dst_ip
    
    global last_remap_idx, flow_map
    flow_tuple = (src_ip, src_port, dst_ip, dst_port)
    if ip_to_map in ips_to_remap_dict:
        if flow_tuple in flow_map:
            ip_to_map = flow_map[flow_tuple]
        else:
            ip_to_map = ip_list[last_remap_idx % len_ip_list]
            flow_map[flow_tuple] = ip_to_map
            last_remap_idx += 1
    return ip_to_map

In [17]:
df["src_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'sa'), axis=1)
df["dst_ip_remapped"] = df.apply(lambda x: remap_ip(x, 'da'), axis=1)
outdir = "../data/remapped_data/"
df.to_csv("../data/remapped_data/remapped_data.csv")

### Data splitting function

In [45]:
import numpy as np
rng = np.random.default_rng()

In [105]:
# Artificial label setting
df['Label']=np.zeros(df.shape[0])
df['Label'][-10:-1]=1.0

In [106]:
# Assume binary for now
# List of parameters
num_agents=10
maintain_ratio=True
seq_select=True

In [102]:
def distribute_dataframe(df,num_agents,maintain_ratio,seq_select):
    list_of_dfs=[]
    if maintain_ratio:
        benign_df=df[df['Label']==0.0]
        mal_df=df[df['Label']==1.0]
        benign_bs=int(np.ceil(len(benign_df)/num_agents))
        mal_bs=int(np.ceil(len(mal_df)/num_agents))
        print(benign_bs,mal_bs)
        if seq_select:
            for i in range(num_agents):
                benign_df_curr=benign_df[i*benign_bs:(i+1)*benign_bs]
                mal_df_curr=mal_df[i*mal_bs:(i+1)*mal_bs]
                df_curr=pd.concat([benign_df_curr,mal_df_curr])
                list_of_dfs.append(df_curr)
        else:
            benign_set=np.array(benign_df.index)
            mal_set=np.array(mal_df.index)
            for i in range(num_agents):
                print(benign_set,mal_set)
                if len(benign_set)>benign_bs:
                    ben_indices_curr=np.sort(rng.choice(benign_set,benign_bs,replace=False))
                    print(ben_indices_curr)
                    benign_set=np.setdiff1d(benign_set,ben_indices_curr)
                else:
                    ben_indices_curr=benign_set
                benign_df_curr=benign_df.loc[ben_indices_curr]
                if len(mal_set)>mal_bs:
                    mal_indices_curr=np.sort(rng.choice(mal_set,mal_bs,replace=False))
                    mal_set=np.setdiff1d(mal_set,mal_indices_curr)
                else:
                    mal_indices=mal_set
                mal_df_curr=mal_df.loc[mal_indices_curr]
                df_curr=pd.concat([benign_df_curr,mal_df_curr])
                list_of_dfs.append(df_curr)
    else:
        bs=int(np.ceil(len(df)/num_agents))
        if seq_select:
            for i in range(num_agents):
                df_curr=df[i*bs:(i+1)*bs]
                list_of_dfs.append(df_curr)
        else:
            all_set=np.array(df.index)
            for i in range(num_agents):
                if len(all_set)>bs:
                    indices_curr=np.sort(rng.choice(all_set,bs,replace=False))
                    all_set=np.setdiff1d(all_set,indices_curr)
                else:
                    indices_curr=all_set
                df_curr=df.loc[indices_curr]
                list_of_dfs.append(df_curr)
    
    return list_of_dfs

In [103]:
dist_dfs=distribute_dataframe(df,num_agents,False, False)

In [104]:
dist_dfs[7]

,ts,te,td,sa,da,sp,dp,pr,flg,fwd,stos,ipkt,ibyt,opkt,obyt,in,out,sas,das,smk,dmk,dtos,dir,nh,nhb,svln,dvln,ismc,odmc,idmc,osmc,mpls1,mpls2,mpls3,mpls4,mpls5,mpls6,mpls7,mpls8,mpls9,mpls10,cl,sl,al,ra,eng,exid,tr,st_timestamp,et_timestamp,src_ip_remapped,dst_ip_remapped,Label
22,2021-01-31 15:26:36,2021-01-31 15:26:37,0.922,10.0.0.153,224.0.0.251,5353,5353.00,UDP,........,0.00,0.00,2.00,478.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0.0.0,0.0.0.0,0.00,0.00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.00,0.00,0.00,0.0.0.0,0/0,0.00,1969-12-31 18:00:00.000,1612128396.00,1612128397.00,10.0.0.153,224.0.0.251,0.00
23,2021-01-31 15:26:31,2021-01-31 15:26:51,20.164,34.203.91.157,10.0.0.182,443,51479.00,TCP,...AP...,0.00,0.00,5.00,272.00,6.00,420.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0.0.0,0.0.0.0,0.00,0.00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,1996.93,81.82,36.70,0.0.0.0,0/0,0.00,1969-12-31 18:00:00.000,1612128391.00,1612128411.00,34.203.91.157,10.0.0.182,0.00
32,2021-01-31 15:26:25,2021-01-31 15:26:25,0.038,2601:246:d00:3510:4d5:6b14:15db:7d3b,2001:558:feed::2,11086,53.00,UDP,........,0.00,0.00,1.00,33.00,1.00,49.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0.0.0,0.0.0.0,0.00,0.00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.00,0.00,0.00,0.0.0.0,0/0,0.00,1969-12-31 18:00:00.000,1612128385.00,1612128385.00,2601:246:d00:3510:4d5:6b14:15db:7d3b,2001:558:feed::2,0.00
46,2021-01-31 15:26:31,2021-01-31 15:26:51,20.168,34.203.91.157,10.0.0.182,443,51477.00,TCP,...AP...,0.00,0.00,5.00,272.00,6.00,420.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0.0.0,0.0.0.0,0.00,0.00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,816.84,84.50,38.84,0.0.0.0,0/0,0.00,1969-12-31 18:00:00.000,1612128391.00,1612128411.00,34.203.91.157,10.0.0.182,1.00
47,2021-01-31 15:26:43,2021-01-31 15:26:43,0.000,fe80::14f7:20ff:fecc:e19a,2601:246:d00:3510:4d5:6b14:15db:7d3b,0,34560.00,ICMP6,........,0.00,0.00,1.00,32.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0.0.0,0.0.0.0,0.00,0.00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,00:00:00:00:00:00,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0-0-0,0.00,0.00,0.00,0.0.0.0,0/0,0.00,1969-12-31 18:00:00.000,1612128403.00,1612128403.00,fe80::14f7:20ff:fecc:e19a,2601:246:d00:3510:4d5:6b14:15db:7d3b,1.00
